#Introduction


#1. Setup

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import math
from zipfile import ZipFile
from urllib.request import urlretrieve

import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.layers import StringLookup

# 2. Load dataset

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdriver')
path = '/content/gdriver/My Drive/Session based recommendation in fashion/dressipi_recsys2022_dataset'
os.chdir(path)
os.listdir()

Mounted at /content/gdriver


['candidate_items.csv',
 'test_final_sessions.csv',
 'test_full_purchases.csv',
 'test_final_purchases.csv',
 'item_features.csv',
 'test_leaderboard_purchases.csv',
 'test_leaderboard_sessions.csv',
 'test_full_sessions.csv',
 'train_purchases.csv',
 'train_sessions.csv',
 'README.txt',
 'dressipi_data_train0.2',
 'train_data_test.csv',
 'test_data_test.csv',
 'train_data_full.csv',
 'test_data_full.csv']

In [ ]:
!ls

candidate_items.csv	test_final_purchases.csv	train_data_full.csv
dressipi_data_train0.2	test_final_sessions.csv		train_data_test.csv
item_features.csv	test_full_purchases.csv		train_purchases.csv
README.txt		test_full_sessions.csv		train_sessions.csv
test_data_full.csv	test_leaderboard_purchases.csv
test_data_test.csv	test_leaderboard_sessions.csv


In [ ]:
import os
import pandas as pd
from datetime import datetime
import time

train_sessions = pd.read_csv('train_sessions.csv')
candidate_items = pd.read_csv('candidate_items.csv')
train_purchases = pd.read_csv('train_purchases.csv')
item_features = pd.read_csv('item_features.csv')

***Description for dataset train_session (session_id, item_id, date).***   
*   The items that were viewed in a session. A session is equal to a day, so a session is one user's activity on one day.
*   The session is not including the first time the user viewed the item that they bought in the end.
*   The last item in the session will be the last item viewed before viewing the item that they bought.





In [ ]:
train_sessions.head(5)

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [ ]:
train_sessions.shape

(4743820, 3)

***Description for candidate_items***

Refer the the candidate items to recommend from. This is the same list for all test sets.

In [ ]:
candidate_items.head(5)

,item_id
0,4
1,8
2,9
3,19
4,20


***Description for train_purchases (session_id, item_id, date)***
It happended at the end of the session. One purchase per session

In [ ]:
train_purchases.head(5)

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


In [ ]:
#how many train_purchases session?
train_purchases.shape

(1000000, 3)

Nearly just 1/4 sessions user end up and buy something

***Description for item_features (item_id, feature_category_id, feature_value_id)*** refer to the label data of items:
*   feature_category_id: aspect of the item such as 'color'
*   feature_value_id: the value for that aspect

Note that some items may not share many feature_category_ids if they are different types of items.



In [ ]:
item_features.head(5)

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75


In [ ]:
item_features.shape

(471751, 3)

In [ ]:
train_sessions.dtypes

session_id     int64
item_id        int64
date          object
dtype: object

In [ ]:
#convert date to timestamp
train_sessions['date'] = pd.to_datetime(train_sessions['date'])
train_sessions['timestamp'] = train_sessions['date'].map(datetime.timestamp)
train_sessions.head(5)

,session_id,item_id,date,timestamp
0,3,9655,2020-12-18 21:25:00.373,1.608327e+09
1,3,9655,2020-12-18 21:19:48.093,1.608326e+09
2,13,15654,2020-03-13 19:35:27.136,1.584128e+09
3,18,18316,2020-08-26 19:18:30.833,1.598470e+09
4,18,2507,2020-08-26 19:16:31.211,1.598469e+09


In [ ]:
#convert date in train_purchases to timestamp
train_purchases['date'] = pd.to_datetime(train_purchases['date'])
train_purchases['timestamp'] = train_purchases['date'].map(datetime.timestamp)
train_purchases.head(5)

,session_id,item_id,date,timestamp
0,3,15085,2020-12-18 21:26:47.986,1.608327e+09
1,13,18626,2020-03-13 19:36:15.507,1.584128e+09
2,18,24911,2020-08-26 19:20:32.049,1.598470e+09
3,19,12534,2020-11-02 17:16:45.920,1.604337e+09
4,24,13226,2020-02-26 18:27:44.114,1.582742e+09


In [ ]:
train_sessions03 = train_sessions.sample(frac=0.03)
train_sessions03.shape

(142315, 4)

#3. Prepare the data
* Download and prepare the DataFrames
* Transform the data into sequences
* Define metadata
* Create dataset for training and evaluation
* Create model inputs (conver to tensor for model inputs)




## 3.1 Download and prepare the DataFrames

##Transform the data into sequences
*   group by session_id
*   session_id - item_id - positional encoding (pos(vi) = t(vt) - t(vi) - timestamp



In [ ]:
#train_sessions_group = train_sessions.sort_values(by=['session_id', 'timestamp']).groupby('session_id')
#train_sessions_group.head(5)
#group dang object

In [ ]:
#khong groupby, chi sort_values
train_sessions_group = train_sessions03.sort_values(by=['session_id', 'timestamp'], ascending = True)
train_sessions_group.head(5)

,session_id,item_id,date,timestamp
0,3,9655,2020-12-18 21:25:00.373,1.608327e+09
5,18,4026,2020-08-26 19:15:47.232,1.598469e+09
3,18,18316,2020-08-26 19:18:30.833,1.598470e+09
44,36,26536,2020-06-21 10:29:51.337,1.592735e+09
60,77,3173,2020-09-20 20:06:13.217,1.600632e+09


In [ ]:
#same session_id, calculate the duration user stay to view item by the next timestamp - current timestamp

def item_view_duration(data_sorted):
  #Initialize an empty list to store the calculated durations
  durations = []
  #data_sorted['duration'] = 0
  current_session_id = data_sorted.iloc[0]['session_id']

  #Iterate throught the dataset that sorted:
  for i in range(1, len(data_sorted)):
    previous_row = data_sorted.iloc[i - 1]
    current_row = data_sorted.iloc[i]
    if current_session_id == current_row['session_id']:
      #Calculate the duration of view for the previous item
      duration = current_row['timestamp'] - previous_row['timestamp']
      duration = round(duration)

      #Append the duration to row:
      data_sorted.loc[i-1, 'duration'] = duration
      #data_sorted[i-1]['duration'] = duration

    else:
      current_session_id = current_row['session_id']

  return data_sorted

###Apply to 20% random of this dataset to test

In [ ]:
# from sklearn.model_selection import train_test_split
# train_sessions_group = train_test_split(train_sessions_group, test_size=0.03, shuffle=False)

In [ ]:
#train_sessions_group_sample.shape

In [ ]:
train_sessions_duration = item_view_duration(train_sessions_group)

In [ ]:
train_sessions_duration.head(10)

,session_id,item_id,date,timestamp,duration
0,3.0,9655.0,2020-12-18 21:25:00.373,1.608327e+09,NaN
5,18.0,4026.0,2020-08-26 19:15:47.232,1.598469e+09,NaN
3,18.0,18316.0,2020-08-26 19:18:30.833,1.598470e+09,NaN
44,36.0,26536.0,2020-06-21 10:29:51.337,1.592735e+09,NaN
60,77.0,3173.0,2020-09-20 20:06:13.217,1.600632e+09,NaN
75,108.0,11386.0,2020-06-06 17:00:15.067,1.591463e+09,NaN
150,195.0,24641.0,2021-01-05 12:03:32.745,1.609848e+09,NaN
184,248.0,25270.0,2020-12-14 12:45:10.492,1.607950e+09,NaN
204,306.0,3422.0,2021-03-15 11:05:06.911,1.615806e+09,NaN
244,376.0,20048.0,2020-01-25 11:36:54.066,1.579952e+09,NaN


In [ ]:
train_sessions_duration['duration'] = train_sessions_duration['duration'].fillna(1)

In [ ]:
train_sessions_duration['duration'].min()

0.0

In [ ]:
train_sessions_duration['duration'].max()

84403.0

In [ ]:
#Transform the train_sessions into data sequences
train_sessions_group_duration = train_sessions_duration.sort_values(by = 'timestamp').groupby('session_id')
train_sessions_seq = pd.DataFrame(
  data = {
      "session_id": list(train_sessions_group_duration.groups.keys()),
      "item_id": list(train_sessions_group_duration.item_id.apply(list)),
      "duration": list(train_sessions_group_duration.duration.apply(list)),
      "timestamps": list(train_sessions_group_duration.timestamp.apply(list))
  }
)
train_sessions_seq.head(5)

,session_id,item_id,duration,timestamps
0,3.0,[9655.0],[1.0],[1608326700.373]
1,18.0,"[4026.0, 18316.0]","[1.0, 1.0]","[1598469347.232, 1598469510.833]"
2,36.0,[26536.0],[1.0],[1592735391.337]
3,77.0,[3173.0],[1.0],[1600632373.217]
4,108.0,[11386.0],[1.0],[1591462815.067]


## Sequences of a fixed length

In [ ]:
sequence_length = 4
step_size = 2


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


train_sessions_seq.item_id = train_sessions_seq.item_id.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

train_sessions_seq.duration = train_sessions_seq.duration.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)
del train_sessions_seq["timestamps"]

In [ ]:
create_sequences(train_sessions_seq, sequence_length, step_size)

[   session_id item_id duration
 0         3.0      []       []
 1        18.0      []       []
 2        36.0      []       []
 3        77.0      []       [],
    session_id item_id duration
 2        36.0      []       []
 3        77.0      []       []
 4       108.0      []       []
 5       195.0      []       [],
    session_id item_id duration
 4       108.0      []       []
 5       195.0      []       []
 6       248.0      []       []
 7       306.0      []       [],
    session_id item_id duration
 6       248.0      []       []
 7       306.0      []       []
 8       376.0      []       []
 9       384.0      []       [],
     session_id item_id duration
 8        376.0      []       []
 9        384.0      []       []
 10       443.0      []       []
 11       486.0      []       [],
     session_id item_id duration
 10       443.0      []       []
 11       486.0      []       []
 12       512.0      []       []
 13       526.0      []       [],
     session_id item_id 

In [ ]:
train_sessions_item = train_sessions_seq[["session_id", "item_id"]].explode("item_id", ignore_index = True)
train_sessions_item.head(5)

,session_id,item_id
0,3.0,NaN
1,18.0,NaN
2,36.0,NaN
3,77.0,NaN
4,108.0,NaN


In [ ]:
train_sessions_duration_seq = train_sessions_seq[["duration"]].explode("duration", ignore_index = True)
train_sessions_duration_seq.head(5)

,duration
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [ ]:
train_sessions_transformed = pd.concat([train_sessions_item, train_sessions_duration_seq], axis = 1 )


In [ ]:
train_sessions_transformed.head(5)

,session_id,item_id,duration
0,3.0,NaN,NaN
1,18.0,NaN,NaN
2,36.0,NaN,NaN
3,77.0,NaN,NaN
4,108.0,NaN,NaN


In [ ]:
train_sessions_transformed = train_sessions_transformed.dropna()

In [ ]:
train_sessions_transformed["item_id"].head(5)

90      [19611.0, 9342.0, 2447.0, 23906.0]
91      [19611.0, 9342.0, 2447.0, 23906.0]
396    [12804.0, 9862.0, 24990.0, 17429.0]
397    [12804.0, 9862.0, 24990.0, 17429.0]
701    [17804.0, 12574.0, 21130.0, 3856.0]
Name: item_id, dtype: object

In [ ]:
train_sessions_transformed.item_id = train_sessions_transformed.item_id.apply(
    lambda x: ",".join([str(v) for v in x]))

train_sessions_transformed.duration = train_sessions_transformed.duration.apply(
    lambda x: ",".join([str(v) for v in x]))

In [ ]:
train_sessions_transformed.head(5)

,session_id,item_id,duration
90,3573.0,"19611.0,9342.0,2447.0,23906.0","1325.0,1.0,1.0,1.0"
91,3573.0,"19611.0,9342.0,2447.0,23906.0","1325.0,1.0,1.0,1.0"
396,14162.0,"12804.0,9862.0,24990.0,17429.0","1.0,1.0,1.0,217.0"
397,14162.0,"12804.0,9862.0,24990.0,17429.0","1.0,1.0,1.0,217.0"
701,26222.0,"17804.0,12574.0,21130.0,3856.0","1.0,1.0,1.0,1.0"


In [ ]:
train_sessions_transformed.rename(columns = {"item_id": "sequence_item_ids", "duration": "sequence_durations"}, inplace = True)

In [ ]:
train_sessions_transformed.head(5)

,session_id,sequence_item_ids,sequence_durations
90,3573.0,"19611.0,9342.0,2447.0,23906.0","1325.0,1.0,1.0,1.0"
91,3573.0,"19611.0,9342.0,2447.0,23906.0","1325.0,1.0,1.0,1.0"
396,14162.0,"12804.0,9862.0,24990.0,17429.0","1.0,1.0,1.0,217.0"
397,14162.0,"12804.0,9862.0,24990.0,17429.0","1.0,1.0,1.0,217.0"
701,26222.0,"17804.0,12574.0,21130.0,3856.0","1.0,1.0,1.0,1.0"


##Train and test splits (85% and 15%)

In [ ]:
####train_test_split 85% and 15%
random_selection = np.random.rand(len(train_sessions_transformed.index)) <= 0.85
train_data = train_sessions_transformed[random_selection]
test_data = train_sessions_transformed[~random_selection]

train_data.to_csv("train_data_full.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data_full.csv", index=False, sep="|", header=False)

In [ ]:
test_data_test = pd.read_csv('test_data_full.csv')

In [ ]:
test_data_test.head(5)

,3573.0|19611.0,9342.0,2447.0,23906.0|1325.0,1.0,1.0.1,1.0.2
0,55923.0|20028.0,23494.0,1644.0,15158.0|1.0,1.0,1.0,1.0
1,59147.0|15921.0,18503.0,12724.0,15173.0|1.0,1.0,48.0,1.0
2,59147.0|15921.0,18503.0,12724.0,15173.0|1.0,1.0,48.0,1.0
3,74170.0|13756.0,21725.0,13344.0,23440.0|38.0,1.0,1.0,1.0
4,74170.0|13344.0,23440.0,19463.0,24985.0|1.0,1.0,1.0,1.0


##Define metadata

In [ ]:
CSV_HEADER = list(train_sessions_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "session_id": list(train_sessions.session_id.unique()),
    "item_id": list(item_features.item_id.unique()),
    "item_category": list(item_features.feature_category_id.unique()),
    "item_category_value": list(item_features.feature_value_id.unique())
}

ITEM_FEATURES = ["item_category", "item_category_value"]

In [ ]:
CSV_HEADER

['session_id', 'sequence_item_ids', 'sequence_durations']

Create tf.data.Dataset for training and evaluation

In [ ]:
#Get data from csv after saved
def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        item_ids_string = features["sequence_item_ids"]
        sequence_item_ids = tf.strings.split(item_ids_string, ",").to_tensor()

        # The last item id in the sequence is the target item.
        features["target_item_id"] = sequence_item_ids[:, -1]
        features["sequence_item_ids"] = sequence_item_ids[:, :-1]

        duration_string = features["sequence_durations"]
        sequence_durations = tf.strings.to_number(
            tf.strings.split(duration_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_durations[:, -1]
        features["sequence_durations"] = sequence_durations[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

In [ ]:
train_dataset = get_dataset_from_csv("train_data_test.csv", shuffle=True, batch_size=265)

In [ ]:
train_dataset

<_MapDataset element_spec=(OrderedDict([('session_id', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('sequence_item_ids', TensorSpec(shape=(None, None), dtype=tf.string, name=None)), ('sequence_durations', TensorSpec(shape=(None, None), dtype=tf.float32, name=None)), ('target_item_id', TensorSpec(shape=(None,), dtype=tf.string, name=None))]), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

#Create model inputs

Convert to a Keras tensor. A tensor-like object - the fundamental data structure used in TensorFlow and represents multi-dimensional arrays of a specific data type. This allow me to build a Keras model just by knowing the inputs and outputs of the model

In [ ]:
def create_model_inputs(): #to transform tensor-like object
    return {
        "session_id": keras.Input(name="session_id", shape=(1,), dtype="string"),
        "sequence_item_ids": keras.Input(
            name="sequence_item_ids", shape=(sequence_length - 1,), dtype="string"
        ),
        "target_item_id": keras.Input(
            name="target_item_id", shape=(1,), dtype="string"
        ),
        "sequence_durations": keras.Input(
            name="sequence_durations", shape=(sequence_length - 1,), dtype=tf.float32
        ),
    }

# 4. Encode input features

For embedding layers in transformers session-based recommendation system is to embeds all input features into a fixed-size low-dimensional vectors.

*   I. Features to represent an item: includes sequence item features (item_id and category_id) and Positional features. Concatenate to create embedding matrix
*   II. Other features: includes item features, context or different features

Step:
1.   Item in sessions encoder - create an embedding layer
2.   Item in targeted encoder - create an embedding layer
3.   Positional embedding
4.   Construct the transformer input (append I and II)









In [ ]:
!pip install -q keras-core
import keras_core as keras_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.4 MB/s eta 0:00:00
Using TensorFlow backend


In [ ]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471751 entries, 0 to 471750
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   item_id              471751 non-null  int64
 1   feature_category_id  471751 non-null  int64
 2   feature_value_id     471751 non-null  int64
dtypes: int64(3)
memory usage: 10.8 MB


In [ ]:
#filter a subset of item_features table just from item from data train
train_sessions_duration = train_sessions_duration[train_sessions_duration["item_id"].notnull()]
item_features_filtered = item_features.loc[train_sessions_duration["item_id"]]

In [ ]:
item_features_filtered

,item_id,feature_category_id,feature_value_id
9655,587,15,584
4026,248,50,317
18316,1103,5,605
26536,1611,11,859
3173,193,72,557
...,...,...,...
27079,1640,68,106
16347,984,26,507
16283,981,50,240
21630,1308,72,7


In [ ]:
item_features_filtered = item_features_filtered.drop(columns = ['feature_value_id'])

In [ ]:
#create one-hot encoding for items(feature_category_id)
one_hot_item_features_filtered = pd.get_dummies(item_features_filtered, prefix = 'feature_', columns = ['item_id', 'feature_category_id'])


In [ ]:
one_hot_item_features_filtered

,feature__2,feature__3,feature__4,feature__7,feature__8,feature__9,feature__10,feature__11,feature__13,feature__14,...,feature__64,feature__65,feature__66,feature__67,feature__68,feature__69,feature__70,feature__71,feature__72,feature__73
9655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27079,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
16347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:

def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features= False,
    include_item_features= False,
):
    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices.
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=1)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions.
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations.
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create an item embedding encoder
    item_vocabulary = tf.strings.as_string(CATEGORICAL_FEATURES_WITH_VOCABULARY["item_id"]) #convert to string
    item_embedding_dims = int(math.sqrt(len(item_vocabulary)))
    # Create a lookup to convert string values to integer indices.
    item_index_lookup = StringLookup(
        vocabulary=item_vocabulary,
        mask_token=None,
        num_oov_indices=1,
        name="item_index_lookup",
    )
    # Create an embedding layer with the specified dimensions.
    item_embedding_encoder = layers.Embedding(
        input_dim=len(item_vocabulary),
        output_dim=item_embedding_dims,
        name=f"item_embedding",
    )
    # Create a vector lookup for item category.
    ##cate_vectors present one-hot-encoding (matrix 0 and 1 for item_id and feature_category_id)
    ##input_dim presents a len of unique item_id and output_dim presents a len of unique
    #cate_vectors = item_features[feature_category_id_unique].to_numpy()
    cate_vectors = one_hot_item_features_filtered.to_numpy()
    # Create a lookup to convert string values to integer indices.
    item_cate_lookup = layers.Embedding(
        input_dim=cate_vectors.shape[0],
        output_dim=cate_vectors.shape[1],
        embeddings_initializer=keras.initializers.Constant(cate_vectors),
        trainable=False,
        name="cate_vector",
    )
    # Create a processing layer for cates.
    item_embedding_processor = layers.Dense(
        units=item_embedding_dims,
        activation="relu",
        name="process_item_embedding_with_cate",
    )

    ## Define a function to encode a given item id.
    def encode_item(item_id):
        # Convert the string input values into integer indices.
        item_idx = item_index_lookup(item_id)
        item_embedding = item_embedding_encoder(item_idx)
        encoded_item = item_embedding
        if include_item_features:
            item_cate_vector = item_cate_lookup(item_idx)
            encoded_item = item_embedding_processor(
                layers.concatenate([item_embedding, item_cate_vector])
            )
        return encoded_item

    ## Encoding target_item_id
    target_item_id = inputs["target_item_id"]
    encoded_target_item = encode_item(target_item_id)

    ## Encoding sequence item_ids.
    sequence_item_ids = inputs["sequence_item_ids"]
    encoded_sequence_items = encode_item(sequence_item_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=item_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    sequence_durations = inputs["sequence_durations"]
    sequence_durations = keras_core.ops.expand_dims(sequence_durations, -1)
    # Add the positional encoding to the item encodings and multiply them by duration.
    encoded_sequence_items_with_position_and_durations = layers.Multiply()(
        [(encoded_sequence_items + encodded_positions), sequence_durations]
    )

    # Construct the transformer inputs.
    for i in range(sequence_length - 1):
        feature = encoded_sequence_items_with_position_and_durations[:, i, ...]
        feature = keras_core.ops.expand_dims(feature, 1)
        encoded_transformer_features.append(feature)
    encoded_transformer_features.append(encoded_target_item)

    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )

    return encoded_transformer_features, encoded_other_features


#Create a BST Model

In [ ]:
include_user_id = False
include_user_features = False
include_item_features = False

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_item_features
    )

    # Create a multi-headed attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block.
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

#Run training and evaluation experiment

Note: change the evaluation

In [ ]:
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data_full.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5)



Epoch 1/5
5/5 [==============================] - 3s 106ms/step - loss: 5582.7837 - mean_absolute_error: 4.7513
Epoch 2/5
5/5 [==============================] - 0s 72ms/step - loss: 5599.4404 - mean_absolute_error: 5.7335
Epoch 3/5
5/5 [==============================] - 0s 81ms/step - loss: 5587.9058 - mean_absolute_error: 5.5521
Epoch 4/5
5/5 [==============================] - 0s 80ms/step - loss: 5565.8765 - mean_absolute_error: 5.6309
Epoch 5/5
5/5 [==============================] - 0s 83ms/step - loss: 5573.2217 - mean_absolute_error: 6.1070


In [ ]:
# Read the test data.
test_dataset = get_dataset_from_csv("test_data_full.csv", batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

Test MAE: 136.77
